In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import seaborn as sns
import matplotlib.pyplot as plt

path = "./transactions-ser"

all_files = glob.glob(os.path.join(path, "*.csv"))

all_df = []
for f in all_files:
    df = pd.read_csv(f, sep=';')
    df['file'] = f.split('/')[-1]
    all_df.append(df)

data = pd.concat(all_df, ignore_index=True)
merged_df = data.copy()
display(merged_df)

# Récupèrer que les lignes qui contient appartement et maison
df = merged_df[(merged_df.type_local == 'Maison') | (merged_df.type_local == 'Appartement')]

to_drop = [column for column in df.columns if df[column].nunique() == 1]

df1 = df.drop(columns=to_drop)
#df1.drop(['file', 'type_local', 'code_postal', 'nature_culture_speciale','adresse_nom_voie','id_mutation','date_mutation'], axis=1, inplace=True)


df1=df1.replace(to_replace='None', value=np.nan)

missing_rate = df1.isna().sum()/df1.shape[0]
missing_rate

#Supprimer les colonnes ayant un messing_rate > 0.8
df1.drop(columns=['adresse_suffixe','lot1_numero', 'lot1_surface_carrez', 'lot2_numero','lot2_surface_carrez','lot3_numero','lot3_surface_carrez','code_nature_culture_speciale','nature_culture_speciale'], inplace=True)
#Supprimer les colonnes ayant d'autres colonnes les représentant avec des valeur (int, ....)
df1.drop(columns = ['adresse_nom_voie','nature_culture','file','id_mutation','code_postal','type_local','id_parcelle','adresse_code_voie'], inplace=True)
df1 = df1.dropna() #We can drop all the rows that have NaN, we don't lose a lot of data

display(df1)
df1.hist(figsize=(15,8))

sns.heatmap(df1.corr(), annot=True)

df1['annee_mutation'] = df1['date_mutation'].apply(lambda x: x.split('-')[0])
df1['annee_mutation']= df1['annee_mutation'].astype(int)
df1 = df1.drop(['date_mutation'], axis=1)

code = {'Vente': 0,
        'Adjudication': 1,
        'Vente en l\'état futur d\'achèvement': 2,
        'S':0,
        'AG':1,
        'J':2,
        'L':3,
        'AB':4,
        '1':1,
        '2':2,
        '000AB': 0, '000AC': 1, '000AD': 2, '000AE': 3, '000AH': 4, '000AI': 5, '000AK': 6, '000AO': 7, '000AP': 8,
        '000AR': 9, '000AS': 10, '000AT': 11, '000AV': 12, '000AW': 13, '000AX': 14, '000AY': 15, '000AZ': 16, '000BC': 17,
        '000BD': 18, '000BE': 19, '000BH': 20, '000BK': 21, '000BL': 22, '000BM': 23, '000BO': 24, '000BP': 25, '000BR': 26,
        '000BS': 27, '000BT': 28, '000BV': 29, '000BW': 30
       }

for col in df1.select_dtypes('object'):
    df1[col] = df1[col].map(code)

#print(df1['section_prefixe'].unique())
df1.hist(figsize=(15,10))

mask = df1.isnull().any(1)
df1 = df1[~mask]
display(df1)

ig, ax = plt.subplots(figsize=(10,8))
sns.heatmap(df1.corr(), annot=True, ax=ax)




def remove_outliers(df):
    """
    Remove outliers from a dataframe for all columns.
    
    Parameters:
    df: Pandas dataframe
    
    Returns:
    df: Dataframe with outliers removed
    """
    for col in df.columns:
        # Calculate IQR
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        # Index dataframe to select rows with non-outlying values
        df = df[~((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR)))]
    
    return df


df1 = remove_outliers(df1)
df1



sns.boxplot(df1['valeur_fonciere'])

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline  import make_pipeline
from sklearn.linear_model import LinearRegression
degree=2
#polyreg=make_pipeline(PolynomialFeatures(degree),LinearRegression())
#polyreg.fit(X,y)

X = df1.drop(columns=['valeur_fonciere', 'code_nature_culture', 'code_type_local', 'adresse_numero','nature_mutation','numero_disposition'], axis=1)
, axis=1)
y = df1['valeur_fonciere']
poly_features = PolynomialFeatures(degree=degree, include_bias = False)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

poly_features = PolynomialFeatures(degree=degree, include_bias = False)
reg = LinearRegression()
X_poly = poly_features.fit_transform(X_train)
X_poly_test = poly_features.fit_transform(X_test)


reg.fit(X_poly,y_train)




from sklearn.metrics import mean_squared_error
predictions = reg.predict(X_poly_test)
mse = mean_squared_error(y_test, predictions)
print(mse)


sns.distplot((y_test, predictions), bins=50)

reg.score(X_poly_test, y_test)

